In [1]:
library("xgboost")
library("Matrix")
library('Ckmeans.1d.dp')
library('lightgbm')



Attaching package: ‘lightgbm’


The following object is masked from ‘package:xgboost’:

    slice




## Read data and process data labels

In [2]:
time_matrix <- matrix(0,ncol = 3, nrow =4)
colnames(time_matrix) <- c("user_time", "system_time", "elapsed_time")
start_time = Sys.time()

In [3]:
data=read.csv('Daily_features_matrix.csv')
data=data[,2:dim(data)[2]]

In [4]:
dim(data)

[1] 4227   23

In [5]:
head(data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,entropy,⋯,diff1_acf10,diff2_acf1,diff2_acf10,length,user_time,system_time,elapsed_time,user_time.1,system_time.1,elapsed_time.1
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,0,1,0.9977307,2.180566e-11,31.265883,4.202931,0.8853252,3.0007563,0.009502084,⋯,0.007626758,-0.5188252,0.2734535,1006,0.02104115,0.02104115,0.02104115,0,0,0
2,1,0,1,0.9753638,2.434650e-09,24.625465,-1.039634,0.8973350,3.0475729,0.211225819,⋯,0.004072618,-0.5203490,0.2801685,1006,0.02027750,0.02027750,0.02027750,0,0,0
3,1,0,1,0.8937784,8.333451e-06,3.804175,4.843726,0.4029370,0.3356327,0.610410200,⋯,0.002531199,-0.4987115,0.2487157,130,0.01750016,0.01750016,0.01750016,0,0,0
4,1,0,1,0.7424974,8.486237e-06,1.762132,6.219671,0.4733951,0.4720488,0.740639635,⋯,0.058828311,-0.5022730,0.3038757,169,0.02131033,0.02131033,0.02131033,0,0,0
5,1,0,1,0.9764692,1.782445e-07,11.344076,0.859238,0.6058021,0.8285215,0.281090710,⋯,0.122022465,-0.4771979,0.3813929,156,0.15410614,0.15410614,0.15410614,0,0,0
6,1,0,1,0.9963228,8.446153e-11,29.701579,10.112177,0.8651846,3.0647427,0.043039068,⋯,0.076311068,-0.5852963,0.4167008,1006,0.01933837,0.01933837,0.01933837,0,0,0


In [6]:
dlist= load('Daily_thetaf_datalist.RData')
datalist=eval(parse(text = dlist ))
res=datalist[[1]]
MASE=res[,,,6]
m=5

In [7]:
dim(MASE)

[1] 4227    5    4

In [8]:
whichmin<-function(x){
    minx=min(x[x>0])
    loc=which(x==minx)[1]-1
    loc
}

meanunique=function(x)
    {
    mean(unique(x))
}

In [9]:
nanum=c()
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    a=apply(count,1,min)
    if(sum(is.na(a))>0)
        {
        nanum=append(nanum,i)
    }
    }

In [10]:
nanum

NULL

In [11]:
realbestmin=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    min_value=apply(count,1,min)
    if (max(min_value,na.rm = TRUE)==0){
        realbestmin[i,]=0
        }
    else
        {
        min_value[is.na(min_value)]=100 
        realbestmin[i,]= whichmin(min_value)
    }
    }

In [12]:
table(realbestmin)

realbestmin
   0    1    2    3    4 
 710  678  371  535 1933 

In [13]:
realbestmean=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    mean_value=apply(count,1,meanunique)
    if (max(mean_value,na.rm = TRUE)==0){
        realbestmean[i,]=0
        }
    else
        {
        mean_value[is.na(mean_value)]=100 
        realbestmean[i,]= whichmin(mean_value)
    }
    }

In [14]:
realbestmean

4
4
4
3
1
4
4
2
2
4
0


In [15]:
head(data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,entropy,⋯,diff1_acf10,diff2_acf1,diff2_acf10,length,user_time,system_time,elapsed_time,user_time.1,system_time.1,elapsed_time.1
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,0,1,0.9977307,2.180566e-11,31.265883,4.202931,0.8853252,3.0007563,0.009502084,⋯,0.007626758,-0.5188252,0.2734535,1006,0.02104115,0.02104115,0.02104115,0,0,0
2,1,0,1,0.9753638,2.434650e-09,24.625465,-1.039634,0.8973350,3.0475729,0.211225819,⋯,0.004072618,-0.5203490,0.2801685,1006,0.02027750,0.02027750,0.02027750,0,0,0
3,1,0,1,0.8937784,8.333451e-06,3.804175,4.843726,0.4029370,0.3356327,0.610410200,⋯,0.002531199,-0.4987115,0.2487157,130,0.01750016,0.01750016,0.01750016,0,0,0
4,1,0,1,0.7424974,8.486237e-06,1.762132,6.219671,0.4733951,0.4720488,0.740639635,⋯,0.058828311,-0.5022730,0.3038757,169,0.02131033,0.02131033,0.02131033,0,0,0
5,1,0,1,0.9764692,1.782445e-07,11.344076,0.859238,0.6058021,0.8285215,0.281090710,⋯,0.122022465,-0.4771979,0.3813929,156,0.15410614,0.15410614,0.15410614,0,0,0
6,1,0,1,0.9963228,8.446153e-11,29.701579,10.112177,0.8651846,3.0647427,0.043039068,⋯,0.076311068,-0.5852963,0.4167008,1006,0.01933837,0.01933837,0.01933837,0,0,0


In [16]:
set.seed(100)
index = sample(2,nrow(data),replace = TRUE,prob=c(0.7,0.3))

In [17]:
train_data=data[index==1,]
test_data=data[index==2,]
train_label_min=realbestmin[index==1,]
test_label_min=realbestmin[index==2,]
train_label_mean=realbestmean[index==1,]
test_label_mean=realbestmean[index==2,]

In [18]:
head(train_data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,entropy,⋯,diff1_acf10,diff2_acf1,diff2_acf10,length,user_time,system_time,elapsed_time,user_time.1,system_time.1,elapsed_time.1
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,0,1,0.9977307,2.180566e-11,31.265883,4.202931,0.8853252,3.0007563,0.009502084,⋯,0.007626758,-0.5188252,0.2734535,1006,0.02104115,0.02104115,0.02104115,0,0,0
2,1,0,1,0.9753638,2.434650e-09,24.625465,-1.039634,0.8973350,3.0475729,0.211225819,⋯,0.004072618,-0.5203490,0.2801685,1006,0.02027750,0.02027750,0.02027750,0,0,0
3,1,0,1,0.8937784,8.333451e-06,3.804175,4.843726,0.4029370,0.3356327,0.610410200,⋯,0.002531199,-0.4987115,0.2487157,130,0.01750016,0.01750016,0.01750016,0,0,0
4,1,0,1,0.7424974,8.486237e-06,1.762132,6.219671,0.4733951,0.4720488,0.740639635,⋯,0.058828311,-0.5022730,0.3038757,169,0.02131033,0.02131033,0.02131033,0,0,0
5,1,0,1,0.9764692,1.782445e-07,11.344076,0.859238,0.6058021,0.8285215,0.281090710,⋯,0.122022465,-0.4771979,0.3813929,156,0.15410614,0.15410614,0.15410614,0,0,0
6,1,0,1,0.9963228,8.446153e-11,29.701579,10.112177,0.8651846,3.0647427,0.043039068,⋯,0.076311068,-0.5852963,0.4167008,1006,0.01933837,0.01933837,0.01933837,0,0,0


In [19]:
end_time = Sys.time()

In [20]:
time_matrix[1,]=end_time-start_time

## Target the interval where the actual error is minimum

In [21]:
start_time = Sys.time()

In [22]:
dtrain_xg_min_reg <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label_min)) 
dtrain_xg_min_cl <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_min)) )

In [23]:
dtrain_lg_min_reg <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label_min))
dtrain_lg_min_cl <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_min)))

In [24]:
xgb_min_reg <- xgboost(data = dtrain_xg_min_reg, nround=100)

[1]	train-rmse:2.008951 
[2]	train-rmse:1.625743 
[3]	train-rmse:1.382650 
[4]	train-rmse:1.228199 
[5]	train-rmse:1.138445 
[6]	train-rmse:1.075843 
[7]	train-rmse:1.037297 
[8]	train-rmse:1.003206 
[9]	train-rmse:0.980319 
[10]	train-rmse:0.965273 
[11]	train-rmse:0.955876 
[12]	train-rmse:0.943480 
[13]	train-rmse:0.933175 
[14]	train-rmse:0.921338 
[15]	train-rmse:0.912649 
[16]	train-rmse:0.884701 
[17]	train-rmse:0.863773 
[18]	train-rmse:0.850635 
[19]	train-rmse:0.838637 
[20]	train-rmse:0.818529 
[21]	train-rmse:0.806558 
[22]	train-rmse:0.793417 
[23]	train-rmse:0.780342 
[24]	train-rmse:0.766782 
[25]	train-rmse:0.748230 
[26]	train-rmse:0.739089 
[27]	train-rmse:0.732863 
[28]	train-rmse:0.713669 
[29]	train-rmse:0.699085 
[30]	train-rmse:0.680070 
[31]	train-rmse:0.667028 
[32]	train-rmse:0.653512 
[33]	train-rmse:0.651106 
[34]	train-rmse:0.646250 
[35]	train-rmse:0.638302 
[36]	train-rmse:0.630660 
[37]	train-rmse:0.624364 
[38]	train-rmse:0.613829 
[39]	train-rmse:0.600

In [25]:
xgb_min_cl <- xgboost(data = dtrain_xg_min_cl, nround=100, objective='multi:softmax',num_class=5)

[1]	train-mlogloss:1.332198 
[2]	train-mlogloss:1.170103 
[3]	train-mlogloss:1.056147 
[4]	train-mlogloss:0.971849 
[5]	train-mlogloss:0.903592 
[6]	train-mlogloss:0.853018 
[7]	train-mlogloss:0.808937 
[8]	train-mlogloss:0.765866 
[9]	train-mlogloss:0.734402 
[10]	train-mlogloss:0.704729 
[11]	train-mlogloss:0.679263 
[12]	train-mlogloss:0.648325 
[13]	train-mlogloss:0.629950 
[14]	train-mlogloss:0.609813 
[15]	train-mlogloss:0.592108 
[16]	train-mlogloss:0.576723 
[17]	train-mlogloss:0.562428 
[18]	train-mlogloss:0.545585 
[19]	train-mlogloss:0.530212 
[20]	train-mlogloss:0.515487 
[21]	train-mlogloss:0.505106 
[22]	train-mlogloss:0.493055 
[23]	train-mlogloss:0.485314 
[24]	train-mlogloss:0.469958 
[25]	train-mlogloss:0.460231 
[26]	train-mlogloss:0.446842 
[27]	train-mlogloss:0.437269 
[28]	train-mlogloss:0.430621 
[29]	train-mlogloss:0.420660 
[30]	train-mlogloss:0.411732 
[31]	train-mlogloss:0.401087 
[32]	train-mlogloss:0.389327 
[33]	train-mlogloss:0.374676 
[34]	train-mlogloss

In [26]:
lgb_min_reg <- lgb.train(data = dtrain_lg_min_reg, nrounds = 100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022318 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4321
[LightGBM] [Info] Number of data points in the train set: 2968, number of used features: 17
[LightGBM] [Info] Start training from score 2.571092


In [27]:
params <- list(objective = "multiclass",
               num_class = 5, 
               metric = "multi_logloss")
lgb_min_cl <- lgb.train(data = dtrain_lg_min_cl,nrounds = 100,params=params)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.026273 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4321
[LightGBM] [Info] Number of data points in the train set: 2968, number of used features: 17
[LightGBM] [Info] Start training from score -1.785044
[LightGBM] [Info] Start training from score -1.873151
[LightGBM] [Info] Start training from score -2.470191
[LightGBM] [Info] Start training from score -2.058107
[LightGBM] [Info] Start training from score -0.762910


In [28]:
end_time = Sys.time()
time_matrix[2,]=end_time-start_time

## Target the interval where the average error is minimum

In [21]:
start_time = Sys.time()

In [22]:
dtrain_xg_mean_reg <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label_mean)) 
dtrain_xg_mean_cl <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_mean)) )

In [23]:
dtrain_lg_mean_reg <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label_mean))
dtrain_lg_mean_cl <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_mean)))

In [24]:
xgb_mean_reg <- xgboost(data = dtrain_xg_mean_reg, nround=100)

[1]	train-rmse:1.834369 
[2]	train-rmse:1.517489 
[3]	train-rmse:1.325189 
[4]	train-rmse:1.197905 
[5]	train-rmse:1.116929 
[6]	train-rmse:1.066559 
[7]	train-rmse:1.033826 
[8]	train-rmse:1.000306 
[9]	train-rmse:0.980132 
[10]	train-rmse:0.958868 
[11]	train-rmse:0.943960 
[12]	train-rmse:0.937575 
[13]	train-rmse:0.919414 
[14]	train-rmse:0.906639 
[15]	train-rmse:0.894477 
[16]	train-rmse:0.879290 
[17]	train-rmse:0.855343 
[18]	train-rmse:0.829354 
[19]	train-rmse:0.812998 
[20]	train-rmse:0.791865 
[21]	train-rmse:0.773618 
[22]	train-rmse:0.760477 
[23]	train-rmse:0.754805 
[24]	train-rmse:0.743268 
[25]	train-rmse:0.726290 
[26]	train-rmse:0.705746 
[27]	train-rmse:0.700372 
[28]	train-rmse:0.690336 
[29]	train-rmse:0.675260 
[30]	train-rmse:0.663513 
[31]	train-rmse:0.658952 
[32]	train-rmse:0.646717 
[33]	train-rmse:0.639846 
[34]	train-rmse:0.631616 
[35]	train-rmse:0.625734 
[36]	train-rmse:0.613046 
[37]	train-rmse:0.607674 
[38]	train-rmse:0.596737 
[39]	train-rmse:0.589

In [25]:
xgb_mean_cl <- xgboost(data = dtrain_xg_mean_cl, nround=100, objective='multi:softmax',num_class=5)

[1]	train-mlogloss:1.384383 
[2]	train-mlogloss:1.238400 
[3]	train-mlogloss:1.139479 
[4]	train-mlogloss:1.054189 
[5]	train-mlogloss:0.990213 
[6]	train-mlogloss:0.936618 
[7]	train-mlogloss:0.887132 
[8]	train-mlogloss:0.841724 
[9]	train-mlogloss:0.808065 
[10]	train-mlogloss:0.779132 
[11]	train-mlogloss:0.743688 
[12]	train-mlogloss:0.722654 
[13]	train-mlogloss:0.706396 
[14]	train-mlogloss:0.687654 
[15]	train-mlogloss:0.664122 
[16]	train-mlogloss:0.645618 
[17]	train-mlogloss:0.626663 
[18]	train-mlogloss:0.609203 
[19]	train-mlogloss:0.598177 
[20]	train-mlogloss:0.586281 
[21]	train-mlogloss:0.570960 
[22]	train-mlogloss:0.560178 
[23]	train-mlogloss:0.546862 
[24]	train-mlogloss:0.533330 
[25]	train-mlogloss:0.515822 
[26]	train-mlogloss:0.507642 
[27]	train-mlogloss:0.497140 
[28]	train-mlogloss:0.484665 
[29]	train-mlogloss:0.469810 
[30]	train-mlogloss:0.455703 
[31]	train-mlogloss:0.444413 
[32]	train-mlogloss:0.434625 
[33]	train-mlogloss:0.426351 
[34]	train-mlogloss

In [ ]:
lgb_mean_reg <- lgb.train(data = dtrain_lg_mean_reg,nrounds = 100)

In [ ]:
params <- list(objective = "multiclass",
               num_class = 5, 
               metric = "multi_logloss")
lgb_mean_cl <- lgb.train(data = dtrain_lg_mean_cl,params=params,nrounds = 100)

In [ ]:
end_time = Sys.time()
time_matrix[3,]=end_time-start_time

## predict

In [37]:
start_time = Sys.time()

In [38]:
alldatalgb <- lgb.Dataset(data = as.matrix(data))
alldataxgb <- xgb.DMatrix(data = as.matrix(data))


In [39]:
xgbregmin=predict(xgb_min_reg,alldataxgb)
xgbclsmin=predict(xgb_min_cl,alldataxgb)
lgbregmin=predict(lgb_min_reg,as.matrix(data))
lgbclsmin=predict(lgb_min_cl,as.matrix(data))

In [40]:
xgbregmin

[1]  3.8057847023  3.6010615826  3.9284517765  3.0956819057  1.2180399895
   [6]  4.0835204124  2.6951041222  2.8709270954  2.7949934006  4.0986757278
  [11]  2.0031177998  0.6598741412  0.3296240866  3.9365017414  3.8281128407
  [16]  0.0834060907  0.3163779080  0.5680813789  3.8824133873  0.0728036091
  [21]  3.9296190739  3.4761178493  3.8822350502  3.4317655563  3.8535578251
  [26]  3.9342222214  3.2636563778  1.2830868959  3.9324073792  0.0204122793
  [31] -0.0870130286  0.7893696427  4.0158939362  3.8101451397  2.2350697517
  [36]  2.4923794270  3.3909890652  3.9224319458  3.3187820911  0.1813727617
  [41]  3.9936285019  0.0943929404  2.7000577450  4.5316739082  0.0604002774
  [46]  3.9243879318  2.0926117897  3.9397039413  2.9497377872  3.9865069389
  [51]  4.0436568260  1.1671720743  0.1383987516  1.2424949408  0.1167584434
  [56]  3.0065729618  2.1967382431  1.2234261036  3.4150204659  3.1250116825
  [61]  2.0879807472  3.9745991230  1.9131160975  0.2885495722  3.6915202141
  [66]  2.9116518497  3.0042386055  3.1478500366  0.0146457516  4.1020750999
  [71]  1.1902431250  0.4085373580  3.1169509888  2.3327105045  2.8523364067
  [76]  3.8633098602  2.4534714222  3.2163407803  2.4714753628  4.0795516968
  [81]  1.0629127026  3.8536283970  1.1021989584  1.2045308352  1.0768253803
  [86]  4.0875487328  3.4746673107  0.1038733274  3.9047489166  4.0867033005
  [91]  3.7869434357  3.7388718128  0.1000025868  0.0534290746  2.1105716228
  [96]  3.9538598061  0.3502902091  2.0015501976  3.8631682396  2.8117501736
 [101]  3.7465996742  3.5755438805  3.7523870468  3.9435489178  3.8694591522
 [106]  2.0047078133  2.2742877007  2.9991405010  4.0414619446  3.6383001804
 [111]  3.2261428833  3.3669002056  3.8586952686  2.9652101994  1.1478575468
 [116]  2.6758348942  3.1940672398  0.0863801688  0.2104268372  3.8204724789
 [121]  3.7983727455  3.7766292095  3.6672413349  3.6168534756  0.2814068496
 [126]  1.0274692774  2.2656686306  2.9297358990  3.9411551952  1.8955748081
 [131]  2.8712937832  4.6909499168  3.1174187660  3.1310155392  3.0413420200
 [136]  3.8422012329  3.9439098835  3.5885198116  0.1821631044  2.0039138794
 [141]  2.8275301456  1.9905536175  2.6106221676  2.0119762421  3.7181546688
 [146]  0.4715372920  0.6164354086  3.9239883423  2.8822920322  1.9564083815
 [151]  2.2483940125 -0.1359152794  3.4912068844  3.8117344379  2.3065853119
 [156]  3.4286952019  3.9214446545  1.1455231905  4.0465669632  1.8149033785
 [161]  1.5892925262  2.1063742638  3.5495085716  0.9992139339  0.9307520390
 [166]  3.8756370544  3.9518182278  2.5572900772  1.9647725821  1.0348379612
 [171]  0.2272452265  0.3573124111  0.3297826946  1.6935861111  2.8683304787
 [176]  3.9264707565  4.5331630707  4.0085582733  0.3531405330  0.4292260706
 [181]  0.3833020926  3.8411133289  3.5543835163  2.9849290848  3.0495204926
 [186]  3.1722965240  3.8956897259  3.9209182262  4.0051536560  2.9973704815
 [191]  3.8568494320  3.1209900379  1.0800511837  3.2922344208  0.3857106864
 [196]  3.7071521282  2.1372826099  2.2544021606  0.9233685136  2.3814537525
 [201]  0.1952744722  3.6657536030  4.0239000320  2.8745131493  1.0550961494
 [206]  2.7754228115  4.1189417839  4.0131669044  4.1255311966  3.8154840469
 [211]  3.8700861931  2.9115986824  0.0686792284  0.1299493909  3.7184746265
 [216]  0.2204632163  1.2971479893  3.6354556084  3.3359172344  3.8491370678
 [221]  0.0826715827  2.2130856514  3.1063544750  3.6238262653  3.3075942993
 [226]  4.0668687820  3.9721839428  3.9505729675  3.8380060196  3.8369472027
 [231]  3.8687219620  4.0289564133  1.5021567345  0.0238988530  3.3659322262
 [236]  3.4217207432  2.6039810181  3.5598621368  2.3911812305  2.0261135101
 [241]  0.6889905334  2.8005306721  2.3931612968  3.0647518635  3.4245901108
 [246]  2.9435789585  2.1468219757  2.6394839287  0.1834223568  0.1755580902
 [251]  0.0904571190  2.2333331108  0.1823314875  0.1688181758  1.4079240561
 [256]  3.7273583412  2.2457389832  2.1385576725  2.2360124588  3.4083044529

In [41]:
xgbclsmin

[1] 4 4 4 3 1 4 4 3 3 4 1 4 0 4 4 0 0 0 4 0 4 4 4 4 4 4 4 0 4 0 0 0 4 4 2 4 4
  [38] 4 4 0 4 0 4 4 0 4 1 4 3 4 4 1 0 1 0 3 2 1 4 3 2 4 4 0 4 3 3 3 0 4 1 0 3 4
  [75] 3 4 4 4 4 4 1 4 1 0 1 4 4 0 4 4 4 4 0 0 4 4 0 2 4 4 4 4 4 4 4 2 0 4 4 4 4
 [112] 4 4 3 4 3 4 0 0 4 4 4 4 4 0 1 4 4 4 2 3 4 4 3 3 4 4 4 0 2 3 2 3 4 4 0 1 4
 [149] 3 2 2 1 4 4 2 4 4 0 4 2 4 2 4 1 1 4 4 4 4 4 0 0 0 4 0 4 4 4 0 0 0 4 4 3 3
 [186] 3 4 4 4 4 4 3 1 0 0 4 2 2 0 4 0 4 4 4 1 4 4 4 4 4 4 4 0 0 4 0 0 4 4 4 0 2
 [223] 3 4 3 4 4 4 4 4 4 4 0 0 3 4 2 4 4 4 0 4 2 4 4 4 2 2 0 0 0 0 0 0 0 4 2 2 2
 [260] 4 4 4 4 4 4 4 0 4 4 4 4 4 4 1 1 4 1 4 4 2 4 4 4 4 4 4 4 4 4 3 4 3 4 2 0 0
 [297] 0 0 4 4 4 0 1 1 1 4 1 1 1 4 2 4 2 4 2 4 4 4 0 4 0 0 0 4 0 0 4 2 2 4 2 4 2
 [334] 4 1 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 0 0 0 0 0 0 2 2 4 4 4
 [371] 4 3 3 3 0 4 0 4 4 4 4 4 4 1 4 1 1 4 0 2 3 4 4 2 2 4 4 4 4 1 4 1 4 4 4 4 4
 [408] 4 4 3 3 4 4 4 0 4 1 4 2 4 4 4 0 4 4 3 0 0 0 0 4 0 4 3 4 4 4 4 1 2 4 3 3 4
 [445] 0 4 4 2 4 0 0 4 4 4 2 4 4 1 4 4 4 4 4 1 4 1 4 4 4 4 4 3 0 1 4 4 4 4 4 4 4
 [482] 1 4 4 1 1 1 4 4 4 4 4 4 0 1 0 4 4 1 4 4 0 4 0 3 4 4 1 0 3 3 0 0 0 0 0 0 0
 [519] 1 0 0 0 2 2 2 2 2 4 4 2 3 4 3 3 4 4 3 3 3 4 2 3 3 1 0 1 1 1 0 4 1 1 1 1 3
 [556] 4 4 0 4 0 4 4 0 4 4 0 4 4 4 4 1 0 0 0 0 0 0 0 0 0 0 0 0 0 2 4 4 4 0 0 4 4
 [593] 4 4 4 0 4 4 3 4 4 4 3 3 4 4 4 4 4 4 4 0 4 0 0 0 0 0 0 0 0 0 0 0 0 4 0 1 1
 [630] 4 2 0 2 2 4 4 4 4 4 2 2 4 4 2 2 2 2 4 4 0 0 0 1 0 3 3 1 0 3 3 3 1 3 3 2 3
 [667] 4 2 3 3 3 4 4 3 4 0 4 1 4 3 4 4 0 4 4 4 4 3 4 4 4 4 4 4 4 4 1 3 4 4 0 4 4
 [704] 4 1 4 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 4 4 4 4 4 3 4 1 3 4 4 4 4 4 1
 [741] 1 1 4 0 1 1 1 1 4 0 0 1 1 1 0 1 0 1 1 0 1 0 1 1 4 0 1 0 4 1 1 4 1 1 4 4 4
 [778] 4 4 1 1 1 0 4 3 4 2 4 3 4 4 4 0 4 0 4 4 2 4 4 3 4 4 4 0 4 4 4 4 4 4 0 4 4
 [815] 4 4 4 4 4 4 4 4 4 1 1 4 0 1 4 1 4 4 4 4 4 3 4 4 4 4 4 4 1 1 2 4 4 3 3 3 4
 [852] 3 0 3 3 4 4 4 3 3 4 3 4 4 4 0 1 1 1 0 1 4 3 0 1 1 1 1 1 1 1 1 1 1 1 2 1 1
 [889] 1 4 1 1 4 0 1 1 1 2 1 1 1 1 4 1 1 0 1 1 1 1 1 1 4 4 4 4 4 3 2 1 4 4 4 4 4
 [926] 4 4 0 4 4 4 4 4 4 4 1 4 4 4 4 4 4 1 1 1 1 1 1 1 1 1 4 0 0 1 0 0 0 0 0 0 0
 [963] 0 0 0 0 0 0 0 0 0 2 1 1 1 1 1 1 1 4 1 0 3 1 1 4 4 4 4 4 3 4 1 4 4 4 4 1 4
[1000] 4 1 3 4 4 4 4 4 1 4 4 4 4 0 4 4 4 4 4 4 4 0 4 4 4 1 0 0 0 4 0 4 2 0 0 0 0
[1037] 3 0 1 1 1 1 4 0 1 4 0 0 4 1 4 3 4 0 2 2 1 0 4 4 1 4 4 0 4 3 0 4 4 4 4 0 4
[1074] 2 4 4 4 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 4 2 4 4 4 3 4 4 4 4 4 4 4 0 4
[1111] 2 0 4 4 3 4 4 4 4 4 4 4 1 3 4 4 2 2 1 1 1 1 2 1 1 2 2 1 1 1 2 1 0 0 1 1 2
[1148] 2 1 2 1 2 0 1 1 0 0 0 1 0 0 2 0 4 0 0 1 0 0 0 0 0 1 0 1 0 4 1 1 1 1 0 1 1
[1185] 1 1 4 4 4 3 1 4 1 1 0 4 4 4 4 1 4 4 4 4 4 4 4 4 4 1 4 4 4 3 0 4 4 4 4 2 4
[1222] 4 4 0 4 4 4 4 4 4 0 0 0 4 1 1 1 4 3 2 4 4 0 0 0 1 4 1 4 4 4 4 4 4 3 4 1 1
[1259] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
[1296] 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 1 1 0 1 0 1 1 1 0 1 0 1 0 1 0 0 0 0 0 0 0 1
[1333] 0 1 1 1 1 1 1 0 0 1 0 4 4 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 3 3 3 3 3 3
[1370] 3 3 3 3 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 4 0 0 0 0 0 4 0 0 0 0
[1407] 0 4 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 3 3 4 4 1 4 4 4 4 4 4 1 1 4
[1444] 1 1 4 1 3 4 1 1 4 4 4 2 4 4 1 1 4 4 0 1 1 0 4 0 4 4 4 4 4 4 0 4 4 4 1 4 4
[1481] 4 4 4 4 4 4 4 4 4 4 4 4 1 1 1 4 4 0 4 0 4 0 0 3 0 3 3 1 3 0 3 0 4 0 4 4 0
[1518] 1 4 4 4 4 4 0 0 0 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[1555] 1 1 1 1 1 1 1 1 1 1 2 1 2 2 1 4 2 1 2 2 4 4 2 1 4 2 2 2 1 1 1 1 1 2 2 4 1
[1592] 2 1 2 2 2 2 2 4 2 2 4 4 4 4 0 4 4 4 3 3 4 4 3 4 0 4 4 4 2 3 4 4 1 4 4 4 4
[1629] 4 4 1 0 4 0 4 4 3 4 4 4 3 4 4 4 4 4 2 4 3 0 2 4 4 4 0 0 1 4 4 4 4 0 4 4 4
[1666] 0 4 4 0 4 4 0 2 1 4 1 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 2 4 4
[1703] 4 3 4 4 4 2 4 3 4 4 1 4 4 4 4 4 3 1 3 4 0 0 4 2 4 4 0 4 4 4 0 4 3 4 4 4 4
[1740] 3 4 4 4 4 2 2 4 4 4 4 4 3 4 4 1 4 2 4 4 4 3 4 3 2 2 4 4 4 3 0 4 4 4 4 0 4
[1777] 0 4 4 4 4 4 4 2 4 4 2 0 4 4 3 0 4 0 4 3 4 4 3 0 0 4 4 4 0 4 4 0 4 4 4 4 0
[1814] 4 4 1 1 4 4 4 2 4 4 4 4 4 4

In [42]:
lgbregmin

[1]  3.9143384005  3.3489003712  3.1401587458  2.7647168138  1.9293060325
   [6]  3.7841400271  2.9771408735  2.5701644328  2.6643022283  3.4193025978
  [11]  2.4077927632  1.4608598233  0.9013542803  3.9375051369  3.8666026011
  [16]  0.8918695215  1.0504392937  2.3893110768  3.8688520201  1.4121171082
  [21]  4.1288669609  3.8879736862  3.4782696022  3.9070339436  3.3004429544
  [26]  3.7556539510  3.0861963665  1.1287079401  4.0227011885  0.8743979611
  [31]  0.5934478427  0.9965218844  3.5940303770  4.1402220687  2.0779573355
  [36]  2.9057651694  2.9830954737  3.5963969903  2.2207742887  1.1545722195
  [41]  3.6444730357  0.9058623060  2.5302151752  2.6018656903  0.8175275313
  [46]  3.4231586135  1.0562725266  3.8844369202  2.7409991676  4.0440862170
  [51]  3.2489546833  1.4626093472  0.4944369993  1.7727451062  1.4286636614
  [56]  3.4001028689  2.4322429175  1.4564059940  3.2550691127  2.9706562838
  [61]  2.1684631017  3.8158302001  2.5921476299  0.6888198162  3.1234035220
  [66]  2.6335313893  2.4208570487  2.7794394017  0.9308464028  3.8635121668
  [71]  1.4566294074  1.1595120966  3.0975989938  2.4392192667  2.7024814433
  [76]  3.9672414098  2.3021290373  2.6749548232  2.6787246092  3.7119031627
  [81]  1.5833991869  3.4245543714  2.0447618401  1.7479454757  1.2751137168
  [86]  3.6314802175  1.7975031402  0.5079027050  2.7978850924  2.7672698464
  [91]  2.2964278791  2.5717600624  0.5155215272  1.0986222139  1.8226539597
  [96]  2.8269632016  1.3589217073  2.6956523256  3.2431637302  2.4624389476
 [101]  3.5229594780  3.3038380838  3.5495857270  3.5387058510  3.6756223621
 [106]  1.6667142882  1.8025383283  3.8054394929  3.8604579251  3.5221629579
 [111]  3.8040761071  3.8683807173  3.7075157134  2.6684793190  1.7833246608
 [116]  2.5911004249  3.6884333088  0.9377901473  0.7331885389  3.7809490380
 [121]  3.6745294563  3.7538479288  3.4209433132  3.2696358418  0.1106711691
 [126]  1.8504854222  2.7223315124  2.4805354388  3.2248815815  1.9897163830
 [131]  2.6242778233  4.1114544971  3.4606298029  3.0003314715  3.5417296382
 [136]  2.8804415485  3.4324183686  3.5411345354  0.9608305370  2.5331745140
 [141]  2.4423730875  2.5361391833  2.8879531134  2.9256007379  3.1743214020
 [146]  0.8793825300  1.5908559714  3.7801144653  2.6459664114  1.7473205385
 [151]  2.4168391039  1.5193345139  3.1154296682  3.6309055974  2.6083376676
 [156]  3.8936955694  4.2801676590  0.4956764476  3.7362145972  1.8201646989
 [161]  3.6049625248  3.0020365041  2.7870207699  0.8802831275  0.8808324610
 [166]  4.1976802267  4.0597095166  2.1446499713  2.3698703999  1.7253732089
 [171]  0.8815054807  0.6379346852  0.7453618264  2.1739974968  2.2427762241
 [176]  3.6079961741  3.8892208104  3.6706893688  0.9644381382  1.3049679078
 [181]  1.6131908227  2.4934471305  3.6315778012  3.4128006477  3.3150003206
 [186]  3.3802642137  3.7659419509  3.4691159141  3.7326512589  3.4476878110
 [191]  3.8042595391  3.3798235329  1.5720066638  3.1680618752  1.7301119724
 [196]  3.0315671984  2.0093609940  2.2743889722  2.0861308912  3.6282999996
 [201]  0.6980208676  2.7158845526  2.8674910728  2.0992538301  1.6875919500
 [206]  3.5326218429  3.8895975990  3.8834490045  3.8815500057  3.7450778416
 [211]  3.7235604909  3.2661633894  1.2001055603  1.2045418997  2.8135114897
 [216]  0.6055493676  1.5903490461  2.4432849038  2.3919183449  2.9723213628
 [221]  0.4813138300  2.4418153204  3.2744476607  3.8182621881  3.5237945270
 [226]  3.6652773353  3.8472308882  3.9921908885  3.6167911364  3.2821380131
 [231]  3.8025303311  3.4530760083  0.6194160142  0.6955306539  3.2574752306
 [236]  3.0862557927  2.7126543210  3.5704934301  3.5256326811  2.5865782663
 [241]  1.6481490276  3.1621167034  2.6904622179  3.3025289753  3.1860288016
 [246]  2.6304876322  2.4428595647  3.0485850736  0.5618215938  0.3768966529
 [251]  0.9891819352  1.3114038520  0.2589607390  0.1905219372  1.1912631551
 [256]  3.0827788363  2.3167899819  2.2784645486  2.4497940817  3.6692586812

In [43]:
lgbclsmin

0.0004831074,0.0002055162,0.0015373814,0.027130537,0.97064346
0.0017210165,0.0184733541,0.0424522055,0.179862285,0.75749114
0.1042212837,0.0016982805,0.0200569761,0.055070835,0.81895262
0.0551661096,0.0064166855,0.0352515641,0.846917964,0.05624768
0.0050526901,0.8332110175,0.0084615359,0.060229711,0.09304505
0.0008345570,0.0001125353,0.0021078210,0.009294862,0.98765023
0.1051445135,0.0094976848,0.2294221224,0.127369515,0.52856616
0.0053798028,0.0112558336,0.0229838385,0.881776283,0.07860424
0.0053060576,0.0605163670,0.0533281672,0.759770134,0.12107927
0.0065352372,0.0088605300,0.0321862565,0.083965653,0.86845232
0.0036736973,0.7171131521,0.0349397912,0.156791403,0.08748196


In [44]:
datalength=dim(data)[1]
lgbclsm=matrix(lgbclsmin,m,datalength)
lgbclsminr=matrix(0,datalength,1)
for(i in 1:datalength)
    {
    lgbclsminr[i,]=which.max(lgbclsm[,i])
}
lgbclsminr=lgbclsminr-1

In [45]:
head(lgbclsminr)

2
1
2
1
3
4


In [46]:
xgbregmean=predict(xgb_mean_reg,alldataxgb)
xgbclsmean=predict(xgb_mean_cl,alldataxgb)
lgbregmean=predict(lgb_mean_reg,as.matrix(data))
lgbclsmean=predict(lgb_mean_cl,as.matrix(data))

In [47]:
lgbclsm=matrix(lgbclsmean,m,datalength)
lgbclsmeanr=matrix(0,datalength,1)
for(i in 1:datalength)
    {
    lgbclsmeanr[i,]=which.max(lgbclsm[,i])
}
lgbclsmeanr=lgbclsmeanr-1

In [48]:
preallmin=cbind(xgbclsmin,xgbregmin)
preallmin=cbind(preallmin,lgbclsminr)
preallmin=cbind(preallmin,lgbregmin)

In [49]:
colnames(preallmin)=c('xgbclsmin','xgbregmin','lgbclsmin','lgbregmin')
head(preallmin)

xgbclsmin,xgbregmin,lgbclsmin,lgbregmin
4,3.805785,2,3.914338
4,3.601062,1,3.348900
4,3.928452,2,3.140159
3,3.095682,1,2.764717
1,1.218040,3,1.929306
4,4.083520,4,3.784140


In [50]:
preallmean=cbind(xgbclsmean,xgbregmean)
preallmean=cbind(preallmean,lgbclsmeanr)
preallmean=cbind(preallmean,lgbregmean)

In [51]:
head(preallmean)

xgbclsmean,xgbregmean,,lgbregmean
4,4.093695,3,3.959947
4,3.770011,1,3.263410
4,3.855083,2,2.997437
3,2.882535,1,2.726431
1,1.073919,4,1.618776
4,3.984329,4,3.846852


In [52]:
colnames(preallmean)=c('xgbclsmean','xgbregmean','lgbclsmean','lgbregmean')
head(preallmean)

xgbclsmean,xgbregmean,lgbclsmean,lgbregmean
4,4.093695,3,3.959947
4,3.770011,1,3.263410
4,3.855083,2,2.997437
3,2.882535,1,2.726431
1,1.073919,4,1.618776
4,3.984329,4,3.846852


In [53]:
end_time = Sys.time()
time_matrix[4,]=end_time-start_time

In [54]:
result_list=list(preallmin,preallmean,time_matrix)

In [55]:
save(result_list, file = "Daily_thetaf_opt_pre_result.RData")

In [56]:
time_matrix

user_time,system_time,elapsed_time
5.307486,5.307486,5.307486
12.788780,12.788780,12.788780
18.910502,18.910502,18.910502
1.144913,1.144913,1.144913


In [29]:
importance_matrix1 <- xgb.importance(model = xgb_mean_cl)
importance_matrix2 <- xgb.importance(model = xgb_mean_reg)
importance_matrix3 <- lgb.importance(lgb_mean_cl)
importance_matrix4 <- lgb.importance(lgb_mean_reg)

importance_matrix1=importance_matrix1[order(importance_matrix1$Feature),]
importance_matrix2=importance_matrix2[order(importance_matrix2$Feature),]
importance_matrix3=importance_matrix3[order(importance_matrix3$Feature),]
importance_matrix4=importance_matrix4[order(importance_matrix4$Feature),]

importance_matrix_mean=(importance_matrix1[,2:4]+importance_matrix2[,2:4]+importance_matrix3[,2:4]+importance_matrix4[,2:4])/4

importance=cbind(importance_matrix1[,1],importance_matrix_mean)

write.csv(importance,'d_thetaf_imp_mean.csv')